# The Price of Art

## Question
What determines the price of (modern) art?

## Hypothesis
An interaction of artist, artwork features, and representation.

## Methodology

1. Examine a model where price is just being predicted by artist, to see if that model can account for significant amounts of variance. 
2. Examine a model where price is just being predicted by artwork features, """ 
3. Examine a model where price is just being predicted by representation, """
4. See if we can do some structural equation modelling of this

# Step 1: Data cleaning

### Import raw data from CSV

In [180]:
import numpy as np
import pandas as pd

# import from raw CSV
artsy_data = pd.read_csv("./data/artworks_list_FINAL.csv", header=0)
artsy_data = artsy_data.drop(columns=['collection_index'])

f"Our raw dataset has {len(artsy_data)} entries."

'Our raw dataset has 2970 entries.'

### Remove error rows
These are entries where we were unable to collect useable data, most likely because the artwork had already been sold at the time of the data scraping, even though it was in the Artsy listings index.

In [181]:
# find rows where error collection has failed
error_rows = artsy_data.index[(artsy_data == "error").any(axis=1)].tolist()

# exclude these rows from the overall dataframe
artsy_listings = artsy_data.drop(error_rows, axis=0)

# how much of original dataframe did we exclude?
rows_excluded = len(error_rows)
percent_excluded = round((len(artsy_data) - len(artsy_listings)) / len(artsy_data) * 100, 2)
f"We excluded {rows_excluded} entries, or {percent_excluded}% of the original dataset, because of errors in data collection."

'We excluded 9 entries, or 0.3% of the original dataset, because of errors in data collection.'

In [182]:
# look at data types
artsy_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2961 entries, 0 to 2969
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   page_url            2961 non-null   object
 1   artist              2961 non-null   object
 2   artist_nationality  2929 non-null   object
 3   artist_birthdate    2002 non-null   object
 4   title               2961 non-null   object
 5   image_url           2961 non-null   object
 6   year                2957 non-null   object
 7   gallery             2960 non-null   object
 8   gallery_location    2960 non-null   object
 9   medium              2961 non-null   object
 10  medium_details      2961 non-null   object
 11  size_inches         2961 non-null   object
 12  size_cm             2961 non-null   object
 13  condition           1341 non-null   object
 14  classification      2961 non-null   object
 15  signed              2821 non-null   object
 16  authenticated       2090

## Clean target variable

The target variable is price, but these are all in different currencies. We will standardise price in GBP.

In [183]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings['price'])

0                                           55,000
1                                            3,500
2                                            2,000
3                                            6,500
5                                            1,250
6                                           15,000
7                                            1,800
8                                              700
9                                            8,000
10                                          11,000
11                                           2,000
12                                           1,000
13                                           3,650
14                                           4,500
15                                           3,200
17                                           2,200
18                                             800
19                                          80,000
20                                           4,500
21                             

In [184]:
# remove all commas from the prices
artsy_listings['price'] = artsy_listings['price'].replace(',','', regex=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings['price'])

0                                            55000
1                                             3500
2                                             2000
3                                             6500
5                                             1250
6                                            15000
7                                             1800
8                                              700
9                                             8000
10                                           11000
11                                            2000
12                                            1000
13                                            3650
14                                            4500
15                                            3200
17                                            2200
18                                             800
19                                           80000
20                                            4500
21                             

In [185]:
# convert all strings into numerics where possible, otherwise fill with NaNs
def convertNumeric(x):
    if type(x) == str:
        if x.isnumeric():
            x = float(x)
        else:
            x = np.NaN
    return x  

# change the price list to the updated values
artsy_listings['price'] = artsy_listings['price'].apply(lambda x: convertNumeric(x))

# take a look and do a summary
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings['price'])

artsy_listings['price'].describe()

0        55000.0
1         3500.0
2         2000.0
3         6500.0
5         1250.0
6        15000.0
7         1800.0
8          700.0
9         8000.0
10       11000.0
11        2000.0
12        1000.0
13        3650.0
14        4500.0
15        3200.0
17        2200.0
18         800.0
19       80000.0
20        4500.0
21        3495.0
22         800.0
23         800.0
24         695.0
25        3500.0
26        1800.0
27        3900.0
28      120000.0
29        2000.0
30       16000.0
31         250.0
32        1900.0
33        1000.0
34        2400.0
35       18000.0
36        6500.0
37       23000.0
38         390.0
39       22000.0
40         250.0
41       13000.0
42        1500.0
43       50000.0
44       12900.0
45        1725.0
46        3200.0
47        1650.0
48        6000.0
49        5500.0
50       22500.0
51       45000.0
52       20000.0
53         200.0
54       10320.0
55       31250.0
56         580.0
57        1000.0
58         800.0
59       17000.0
60        1300

count      2513.000000
mean      14264.425388
std       45704.491797
min          32.000000
25%        2000.000000
50%        4750.000000
75%       12500.000000
max      850000.000000
Name: price, dtype: float64

Now we take a look at currency so we can standardise all these prices into GBP at today's market rate.

In [186]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings[['currency', 'price', 'gallery', 'gallery_location']])

     currency     price                                            gallery  \
0           $   55000.0                              Corridor Contemporary   
1           €    3500.0                                         Make offer   
2           €    2000.0                                          Artistics   
3           $    6500.0                                       Gallery 1261   
5           $    1250.0                                    Signari Gallery   
6           $   15000.0                                        Los Angeles   
7           $    1800.0                                       Gallery 1261   
8           $     700.0                                     Bakker Gallery   
9           €    8000.0                                         Make offer   
10          $   11000.0                          Luis De Jesus Los Angeles   
11          $    2000.0                                           New York   
12          $    1000.0                                       Ga

So we've got a few cases here:

1. Most artwork prices are in USD ($), GBP (£), or EUR (€).
2. We've got two other characters occasionally - "T" and "E" - which mark instances where the information about price was either not available or not in the standard price section. We will exclude these entries.
3. NaN entries - to be excluded.

In [187]:
# What are the rates for USD > GBP and EUR > GBP?
from currency_converter import CurrencyConverter
from datetime import date
c = CurrencyConverter()

# choose a date around when the data was collected
# default is the most recent rate available, but more recent dates in May 2021 are not available yet
c.convert(100, 'EUR', 'GBP', date=date(2021,3,1))
c.convert(100, 'USD', 'GBP', date=date(2021,3,1))

def currencySignConvert(sign, price):
    if sign == "$":
        price_GBP = c.convert(price, 'USD', 'GBP')
    elif sign == "€":
        price_GBP = c.convert(price, 'EUR', 'GBP')
    elif sign == '£':
        price_GBP = price
    else:
        price_GBP = np.NaN

    return price_GBP


## apply function to every row of the dataframe
price_GBP = []
for index, row in artsy_listings.iterrows():
    # print(row['currency'])
    # print(row['price'])
    price_GBP.append(round(currencySignConvert(row['currency'], row['price']),2))
artsy_listings['price_GBP'] = price_GBP

# describe the data
artsy_listings['price_GBP'].describe()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings[['currency', 'price', 'price_GBP']])

     currency     price  price_GBP
0           $   55000.0   39049.86
1           €    3500.0    3017.53
2           €    2000.0    1724.30
3           $    6500.0    4614.98
5           $    1250.0     887.50
6           $   15000.0   10649.96
7           $    1800.0    1278.00
8           $     700.0     497.00
9           €    8000.0    6897.20
10          $   11000.0    7809.97
11          $    2000.0    1420.00
12          $    1000.0     710.00
13          $    3650.0    2591.49
14          $    4500.0    3194.99
15          €    3200.0    2758.88
17          $    2200.0    1561.99
18          $     800.0     568.00
19          $   80000.0   56799.80
20          $    4500.0    3194.99
21          £    3495.0    3495.00
22          €     800.0     689.72
23          €     800.0     689.72
24          €     695.0     599.19
25          €    3500.0    3017.53
26          $    1800.0    1278.00
27          $    3900.0    2768.99
28          $  120000.0   85199.70
29          $    200

Omit rows where there is no price information.

In [188]:
# find rows where there is no price information
error_rows = artsy_listings.index[np.isnan(artsy_listings['price'])]

# exclude these rows from the overall dataframe
artsy_listings = artsy_listings.drop(error_rows, axis=0)

# # how much of original dataframe did we exclude?
rows_excluded = len(error_rows)
percent_excluded = round((len(artsy_data) - len(artsy_listings)) / len(artsy_data) * 100, 2)
f"We excluded {rows_excluded} entries, or {percent_excluded}% of the original dataset, because of errors in data collection."

'We excluded 448 entries, or 15.39% of the original dataset, because of errors in data collection.'

## Clean predictor variables

### 1. Year
Let's start with the artwork side, specifically, **year** (and therefore age) of the artwork.

In [189]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings['year'])

0                                                ca. 2012
1                                                    2017
2                                                    2015
3                                                    2016
5                                                    2019
6                                                    2020
7                                                    2020
8                                            Late 20th c.
9                                                    2017
10                                                     II
11                                                   2018
12                                                   2020
13                                                   2020
14                                                   2020
15                                                   2017
17                                                   2020
18                                                   2020
19            

In [190]:
# check types
# artsy_listings['year'].apply(lambda x: print(type(x)))

# replace any NaNs with none types
artsy_listings['year'] = artsy_listings['year'].replace({np.nan: 'none'})

# check types again
artsy_listings['year'].apply(lambda x: print(type(x)))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

0       None
1       None
2       None
3       None
5       None
        ... 
2962    None
2966    None
2967    None
2968    None
2969    None
Name: year, Length: 2513, dtype: object

In [191]:
# remove 'ca' and 'circa'
artsy_listings['year'] = artsy_listings['year'].replace('ca.','', regex=True)
artsy_listings['year'] = artsy_listings['year'].replace('circa', '', regex=True)
artsy_listings['year'] = artsy_listings['year'].replace('Cir', '', regex=True)

# remove periods
import string
def remove_periods(text):
    text = text.replace('.', '')
    return text

artsy_listings['year'] = artsy_listings['year'].apply(remove_periods)

# find entries containing 1 or 2, otherwise insert "none" (str)
def findYears(entry):
    if "1" not in entry and "2" not in entry:
        entry = "none"
    return entry

artsy_listings['year'] = artsy_listings['year'].apply(findYears)

# replace century with averaged years
def replaceCenturies(entry):
    if "19th" in entry:
        if "early" in entry:
            entry = "1825"
        elif "late" in entry:
            entry = "1875"
        else:
            entry = "1850"
    elif "20th" in entry:
        if "early" in entry:
            entry = "1925"
        elif "late" in entry:
            entry = "1975"
        else:
            entry = "1950"
    elif "21st" in entry:
        entry = "2010"
    elif "contemporary" in entry:
        entry = "2010"
    return entry

artsy_listings['year'] = artsy_listings['year'].apply(replaceCenturies)

# replace split years with later year
def replaceYearSplit(entry):
    delimiter = ["/", "-", "–"]
    for c in delimiter:
        index = entry.find(c)
        entry = entry[index+1:]
    if "to" in entry:
        index = entry.find("to")
        entry = entry[index+3:]
    return entry

artsy_listings['year'] = artsy_listings['year'].apply(replaceYearSplit)

# replace decades ("0s", "0's")
def replaceDecades(entry):
    if "0s" in entry:
        entry = entry.replace("s", "")
        entry = entry.replace("0", "5")
    if "0's" in entry:
        entry = entry.replace("'s", "")
        entry = entry.replace("0", "5")
    return entry

artsy_listings['year'] = artsy_listings['year'].apply(replaceDecades)


# remove 's', 'c', 'cir', extra spaces, and any other punctuation
def removeFaff(entry):
    faff = ["cir", "c", "C", "s", "AD", " "]
    for c in faff:
        entry = entry.replace(c, "")
    return entry
    
artsy_listings['year'] = artsy_listings['year'].apply(removeFaff)


# remove any lines that don't have 4 characters
def noYear(entry):
    if len(entry) != 4:
        entry = "none"
    return entry

artsy_listings['year'] = artsy_listings['year'].apply(noYear)

# convert all years to numerics where possible, otherwise convert to NaNs
def convertYearNumeric(entry):
    if entry == "none":
        return 0
    else:
        return int(entry)

artsy_listings['year'] = artsy_listings['year'].apply(convertYearNumeric)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(artsy_listings['year'])

0       2012
1       2017
2       2015
3       2016
5       2019
6       2020
7       2020
8       1950
9       2017
10         0
11      2018
12      2020
13      2020
14      2020
15      2017
17      2020
18      2020
19      1973
20      2016
21      2020
22      2018
23      2019
24      2020
25      2017
26      2020
27      2017
28      1987
29      2019
30      2020
31      2020
32      2020
33      2009
34      2020
35      2020
36      2020
37         0
38      2020
39      2020
40      1950
41      2020
42      2020
43      2015
44      2020
45      2019
46         0
47      2018
48      2019
49      2018
50      1955
51      2015
52      2020
53      2020
54      2019
55      1994
56      2021
57      2009
58      2019
59      2020
60      2021
61      2018
62      2019
63      1996
64      2017
65         0
66      2011
67      2020
68      1938
69      2020
70      2020
71      2020
72      2015
73      2018
74      2017
75      2020
76      2020
77      2011
78      1999

### Finally, representation variables: gallery and gallery location.

In [162]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(artsy_listings[['gallery', 'gallery_location']])